# 라이브러리

In [1]:
from transformers import ElectraModel, ElectraTokenizer
from transformers import ElectraForSequenceClassification

from transformers import AutoTokenizer, AutoModelForSemanticSegmentation, TrainingArguments, Trainer

import torch

import pandas as pd
from konlpy.tag import Mecab
import re
from tqdm import tqdm, tqdm_notebook

from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report

/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1차 모델

In [2]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
light_data = survey[['content_id', 'light']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
light_data['content_id'] = light_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
light_data = light_data[light_data['content_id'] != 0].reset_index().drop(columns=['index'])
light_data['content_id'] = light_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(light_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
light_data = pd.merge(light_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
light_data['light'] = light_data['light'].astype(int)

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

light_data['content'] = light_data['content'].apply(cleaned_content)

final_df = light_data[['content', 'light']]

# 라벨별 개수 확인
print(final_df['light'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['light'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['light'].value_counts())
print('\n')
# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)

# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))
print('\n')
# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
print('\n')
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_59642/1053599568.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  light_data['content_id'] = light_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


light
 0    4918
-1      95
 1      10
Name: count, dtype: int64


train_data light
 0    3929
-1      81
 1       8
Name: count, dtype: int64

 test_data light
 0    988
-1     17
Name: count, dtype: int64


중복 제거 전 학습 데이터셋: 4018
중복 제거 전 테스트 데이터셋: 1005




중복 제거 후 학습 데이터셋: 4003
중복 제거 후 테스트 데이터셋: 1001


In [ ]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가 
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

In [ ]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["light"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['light'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './light_model/check_point/try_1',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 10,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# train 진행
trainer.train() 

In [ ]:
trainer.save_model("./light_model/try_1/light_model1")
tokenizer.save_pretrained("./light_model/try_1/light_tokenizer1")

In [3]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./light_model/try_1/light_model1"
tokenizer_path = "./light_model/try_1/light_tokenizer1"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    

100%|██████████| 1001/1001 [00:12<00:00, 82.62it/s]


,content,light,pred
0,#1415_20221128_3인. 흐음.잘 열어보지 않으면,0,0
1,난.바보야.,0,0
2,들어가자마자 커피향이 씨게 낫다 좋앗다 하지만 인테리어는 그냥그랫다. 풀정도 되는듯.,0,0
3,3인. 스토리가 잘 감이 안 잡힌다!,0,0
4,1.5/3인. 활동성 있음.추락조심,0,0
...,...,...,...
999,나에겐 너무 어려웠던. 방탈짬바 있는 분들에게 추천,0,0
1000,2021. 12. 26. (3인) 각자 1인분씩 하고 뿌듯했던 테마.,0,0
1001,첫방 디버프로 초반에 절어버림. 볼것도 못보고 힌트썼는데 잘꾸며놓았다 . 살짝 장치...,0,0
1002,#5 - 2인 - 볼륨에 압도됨 - 히터도 방마다 빵빵함 - 다른 테마도 궁금해짐,0,0


In [8]:
print(final_df['light'].value_counts())

light
 0    4918
-1      95
 1      10
Name: count, dtype: int64


In [5]:
print(train_data['light'].value_counts())
print(test_data['light'].value_counts())

light
 0    3915
-1      80
 1       8
Name: count, dtype: int64
light
 0    984
-1     17
Name: count, dtype: int64


In [6]:
print(classification_report(test_data['light'], test_data['pred']))

              precision    recall  f1-score   support

          -1       0.87      0.76      0.81        17
           0       1.00      1.00      1.00       984

    accuracy                           0.99      1001
   macro avg       0.93      0.88      0.90      1001
weighted avg       0.99      0.99      0.99      1001



In [9]:
print(classification_report(test_data['light'], test_data['pred']))
test_data2 = test_data[test_data['light'] != 0]
print(classification_report(test_data2['light'], test_data2['pred']))

              precision    recall  f1-score   support

          -1       0.87      0.76      0.81        17
           0       1.00      1.00      1.00       984

    accuracy                           0.99      1001
   macro avg       0.93      0.88      0.90      1001
weighted avg       0.99      0.99      0.99      1001

              precision    recall  f1-score   support

          -1       1.00      0.76      0.87        17
           0       0.00      0.00      0.00         0

    accuracy                           0.76        17
   macro avg       0.50      0.38      0.43        17
weighted avg       1.00      0.76      0.87        17



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [10]:
test = ['조도 무슨 일', '조도 개별로', '조도 최악', 
'너무 어두워서 별로였어요', '조도가 적당해서 좋았어요', '밝기 적당했음', '공포도 하나도 없음']

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10, '\n')

문장: 조도 무슨 일, 평가: 0
---------- 

문장: 조도 개별로, 평가: 0
---------- 

문장: 조도 최악, 평가: 0
---------- 

문장: 너무 어두워서 별로였어요, 평가: 0
---------- 

문장: 조도가 적당해서 좋았어요, 평가: 0
---------- 

문장: 밝기 적당했음, 평가: 1
---------- 

문장: 공포도 하나도 없음, 평가: 1
---------- 



## 1차 결과
- 긍정 평가 절망적
- 데이터 증강증강증강

# 2차 모델

In [2]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
light_data = survey[['content_id', 'light']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
light_data['content_id'] = light_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
light_data = light_data[light_data['content_id'] != 0].reset_index().drop(columns=['index'])
light_data['content_id'] = light_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(light_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
light_data = pd.merge(light_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
light_data['light'] = light_data['light'].astype(int)
final_df = light_data[['content', 'light']]

# 데이터 추가
new_data = [
    ('조도 적당함', 1), ('밝기 아주 딱이었음', 1), ('적당히 어두워서 좋았어요', 1), ('조도 적당해서 문제도 잘 풀림', 1),
    ('조도 완벽했음', 1), ('온도 습도 밝기 다 완벽했다', 1), ('막 어둡지도 밝지도 않아서 좋았어요', 1), ('적당한 조명 굳', 1),
    ('조도 적당함, 방탈출에서 완벽한 조명', 1), ('밝기 아주 딱이었음, 방탈출 공간에서 최적의 밝기', 1), ('적당히 어두워서 좋았어요, 방탈출시 어둡지도 않아서 편안함', 1),
    ('조도 적당해서 문제도 잘 풀림, 방탈출 게임에서 이상적인 환경', 1), ('조도 완벽했음, 방탈출 룸에서 완벽한 빛 조절', 1),
    ('온도 습도 밝기 다 완벽했다, 방탈출 장소의 모든 조건이 완벽함', 1), ('막 어둡지도 밝지도 않아서 좋았어요, 방탈출에서 딱 좋은 밝기', 1),
    ('적당한 조명 굳, 방탈출에서 필요한 적절한 조명', 1), ('밝기 조절이 좋아 문제 해결이 수월함, 방탈출에서의 조도 조절이 탁월함', 1),
    ('어둡지 않고 밝지도 않아 편안한 분위기, 방탈출에서의 조도가 안성맞춤', 1),

    ('밝기 개쓰레기임', -1), ('조명 실화냐', -1), ('너무 어두워서 하나도 안 보임', -1), ('조명 관리 누가 함?', -1),
    ('너무 밝아서 몰입이 안 됐어요', -1), ('누가 조명가지고 장난치는 듯', -1), ('이건 몰입하라고 설치한 조명일까?', -1), ('조도 최악', -1), 
    ('조도 나쁘다', -1)
]

new_df = pd.DataFrame(new_data, columns=['content', 'light'])
final_df = pd.concat([final_df, new_df])

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

from kiwipiepy import Kiwi
kiwi = Kiwi()

def kiwi_clean(text):
    get_kiwi_pos = ['NNG', 'NP', 'NNP', 'MM', 'VV', 'VV-I', 'VV-R', 'VA', 'VA-I', 'VA-R', 'VCP', 'VCN', 'MAG', 'MAJ', 'XR']
    kiwi_lem = []
    for word in kiwi.tokenize(text):
        if word.tag in get_kiwi_pos:
            kiwi_lem.append(word.lemma)
    return ' '.join(kiwi_lem)

final_df['content'] = final_df['content'].apply(cleaned_content)
final_df['content'] = final_df['content'].apply(kiwi_clean)

final_df = final_df.drop_duplicates()
final_df = final_df.reset_index().drop(columns = ['index'])
final_df

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_74645/1079827769.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  light_data['content_id'] = light_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


,content,light
0,오류 장치 옵으 다시 참여 그 때 문제 못 풀다 이상 쌓이다 생각 잘 품,0
1,무슨 일 이다,-1
2,인 심연 떠나다,0
3,인 극악 조도 그것 다,-1
4,조도 낮다 아니다 없다 수준 이다 하다 확실히 느끼다 조도 낮다 관찰력 문제 많다 ...,-1
...,...,...
4912,너무 밝다 몰입 안 되다,-1
4913,누구 조명 가지다 장난치다,-1
4914,이거 몰입 설치 조명 이다,-1
4915,조 최악,-1


In [5]:

# 라벨별 개수 확인
print(final_df['light'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['light'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['light'].value_counts())
print('\n')

# 데이터셋 개수 확인

light
 0    4786
-1     103
 1      28
Name: count, dtype: int64


train_data light
 0    3827
-1      83
 1      24
Name: count, dtype: int64

 test_data light
 0    940
-1     25
 1     18
Name: count, dtype: int64




In [12]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가 
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(35080, 128)

In [13]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["light"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['light'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './light_model/check_point/try_2',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 30,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [14]:
# train 진행
trainer.train() 

  0%|          | 0/7380 [00:00<?, ?it/s]/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_72084/434806867.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  7%|▋         | 500/7380 [01:28<19:46,  5.80it/s]

{'loss': 0.1682, 'learning_rate': 4.661246612466125e-05, 'epoch': 2.03}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_72084/434806867.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 14%|█▎        | 1000/7380 [02:55<18:23,  5.78it/s]

{'loss': 0.0783, 'learning_rate': 4.3224932249322496e-05, 'epoch': 4.07}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_72084/434806867.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 20%|██        | 1500/7380 [04:23<16:46,  5.84it/s]

...

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_72084/434806867.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 95%|█████████▍| 7000/7380 [20:25<01:05,  5.82it/s]

{'loss': 0.0062, 'learning_rate': 2.574525745257453e-06, 'epoch': 28.46}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_72084/434806867.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 7380/7380 [21:32<00:00,  5.71it/s]

{'train_runtime': 1292.2617, 'train_samples_per_second': 91.328, 'train_steps_per_second': 5.711, 'train_loss': 0.03551455039965105, 'epoch': 30.0}


TrainOutput(global_step=7380, training_loss=0.03551455039965105, metrics={'train_runtime': 1292.2617, 'train_samples_per_second': 91.328, 'train_steps_per_second': 5.711, 'train_loss': 0.03551455039965105, 'epoch': 30.0})

In [15]:
trainer.save_model("./light_model/try_2/light_model2")
tokenizer.save_pretrained("./light_model/try_2/light_tokenizer2")

('./light_model/try_2/light_tokenizer2/tokenizer_config.json',
 './light_model/try_2/light_tokenizer2/special_tokens_map.json',
 './light_model/try_2/light_tokenizer2/vocab.txt',
 './light_model/try_2/light_tokenizer2/added_tokens.json')

In [3]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./light_model/try_2/light_model2"
tokenizer_path = "./light_model/try_2/light_tokenizer2"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    text = kiwi_clean(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    

  0%|          | 0/983 [00:00<?, ?it/s]

100%|██████████| 983/983 [00:11<00:00, 86.35it/s]


,content,light,pred
0,나비효과 시작 뭐 이다 다르다 하다,0,0
1,한 문제 못 푸다 생각 쉽다 문제 두 자물쇠 풀리다 짜증 어떤 문제 당황 19금 요...,0,0
2,바지 추천 무난 마음 몽글몽글 테마 이다 후반부 장치 이슈 시간 지체 넘다 남기다 ...,0,0
3,남친 가다 일 없다 이다 친구 하다 간혹 서로 기분 나쁘다 표정 바라보다 나쁘다 하...,0,0
4,인 장치 오류 아쉽다 재미있다,0,0
...,...,...,...
978,너무 밝다 몰입 안 되다,-1,-1
979,누구 조명 가지다 장난치다,-1,-1
980,이거 몰입 설치 조명 이다,-1,-1
981,조 최악,-1,-1


In [7]:
print(classification_report(test_data['light'], test_data['pred']))
test_data2 = test_data[test_data['light'] != 0]
print(classification_report(test_data2['light'], test_data2['pred']))

              precision    recall  f1-score   support

          -1       0.96      0.88      0.92        25
           0       1.00      1.00      1.00       940
           1       1.00      1.00      1.00        18

    accuracy                           1.00       983
   macro avg       0.98      0.96      0.97       983
weighted avg       1.00      1.00      1.00       983

              precision    recall  f1-score   support

          -1       1.00      0.88      0.94        25
           0       0.00      0.00      0.00         0
           1       1.00      1.00      1.00        18

    accuracy                           0.93        43
   macro avg       0.67      0.63      0.65        43
weighted avg       1.00      0.93      0.96        43



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [8]:
test = ['조도 무슨 일', '조도 개별로', '조도 최악', 
'너무 어두워서 별로였어요', '조도가 적당해서 좋았어요', '밝기 적당했음', '공포도 하나도 없음']

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10, '\n')

문장: 조도 무슨 일, 평가: 2
---------- 

문장: 조도 개별로, 평가: 1
---------- 

문장: 조도 최악, 평가: 0
---------- 

문장: 너무 어두워서 별로였어요, 평가: 0
---------- 

문장: 조도가 적당해서 좋았어요, 평가: 2
---------- 

문장: 밝기 적당했음, 평가: 2
---------- 

문장: 공포도 하나도 없음, 평가: 1
---------- 

